# FER+ CNN

In [1]:
train_dir = '/home/rkuo/Datasets/FERplus/train'
val_dir   = '/home/rkuo/Datasets/FERplus/val'
test_dir  = '/home/rkuo/Datasets/FERplus/test'

## Prepare Dataset

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
batch_size = 8
target_size = (112,112)

train_datagen = ImageDataGenerator(rescale=1./255)
val_datagen   = ImageDataGenerator(rescale=1./255)
test_datagen  = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=True)

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=target_size,
        batch_size=batch_size,
        color_mode="grayscale",
        class_mode='categorical',
        shuffle=False)

2024-04-18 11:15:14.334293: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-18 11:15:14.356060: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-18 11:15:14.658850: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Found 66379 images belonging to 8 classes.
Found 8341 images belonging to 8 classes.
Found 3573 images belonging to 8 classes.


In [3]:
labels = list(train_generator.class_indices.keys())
print(labels)

['angry', 'contempt', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'suprise']


## Build Model

In [4]:
from tensorflow.keras import models, layers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Flatten, Dense

In [5]:
num_classes = len(labels)
input_shape = (112,112,1)

In [6]:
# build model
model=models.Sequential()
model.add(Conv2D(16, kernel_size=(3, 3), activation='relu', padding='same', input_shape=input_shape))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

model.summary()

/home/rkuo/.local/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(
2024-04-18 11:15:15.634356: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 11:15:15.653308: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-04-18 11:15:15.653412: I e

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 112, 112, 16)   │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 56, 56, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 56, 56, 16)     │            64 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 56, 56, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 28, 28, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 28, 28, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 28, 28, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 14, 14, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 14, 14, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 7, 7, 128)      │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 7, 7, 256)      │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 3, 3, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 3, 3, 256)      │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,607,816 (6.13 MB)

 Trainable params: 1,606,824 (6.13 MB)

 Non-trainable params: 992 (3.88 KB)

In [7]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

## Train Model

In [8]:
history = model.fit(train_generator, validation_data=val_generator, epochs=30)

Epoch 1/30


/home/rkuo/.local/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1713410116.866533   24822 service.cc:145] XLA service 0x764204016b50 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1713410116.866581   24822 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Ti, Compute Capability 8.9
2024-04-18 11:15:16.923747: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-04-18 11:15:17.080166: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


 100/8298 ━━━━━━━━━━━━━━━━━━━━ 12s 2ms/step - accuracy: 0.1650 - loss: 2.4542     

I0000 00:00:1713410118.862318   24822 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_4', 36 bytes spill stores, 36 bytes spill loads

I0000 00:00:1713410118.865511   24822 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1271/8298 ━━━━━━━━━━━━━━━━━━━━ 20s 3ms/step - accuracy: 0.2388 - loss: 2.0050

I0000 00:00:1713410122.442372   24820 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_4', 36 bytes spill stores, 36 bytes spill loads



8298/8298 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.3633 - loss: 1.6273 - val_accuracy: 0.5933 - val_loss: 1.2794
Epoch 2/30
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.7439 - loss: 0.7054 - val_accuracy: 0.6383 - val_loss: 1.2731
Epoch 3/30
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8413 - loss: 0.4426 - val_accuracy: 0.6233 - val_loss: 1.7122
Epoch 4/30
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.8864 - loss: 0.3187 - val_accuracy: 0.6492 - val_loss: 1.6443
Epoch 5/30
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9122 - loss: 0.2467 - val_accuracy: 0.6347 - val_loss: 1.9145
Epoch 6/30
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9302 - loss: 0.1972 - val_accuracy: 0.6354 - val_loss: 1.9865
Epoch 7/30
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9444 - loss: 0.1647 - val_accuracy: 0.6545 - val_loss: 2.2461
Epoch 8/30
8298/8298 ━━━━━━━━━━━━━━━━━━━━ 14s 2ms/step - accuracy: 0.9517 - loss: 0.1413 - val

## Evaluate Model

In [9]:
test_loss, test_acc = model.evaluate(test_generator)
print("Test     loss:", test_loss)
print("Test accuracy:", test_acc)

447/447 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7252 - loss: 2.0997
Test     loss: 1.8814784288406372
Test accuracy: 0.7383151650428772


## Save Model

In [10]:
model.save('ferplus_cnn.h5')